In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch as tc
#from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [ ]:
class Morphism(nn.Module):
    def __init__ (self, name = 'Morphisme R^n --> E', dim_E = 1, neurons = 6):
        print(f'[Model] name : {name}')
        print(f'[Model] dim E : {dim_E}')
        print(f'[Model] no. neurons per layers : {neurons}')
        super(Morphism, self).__init__()
        # layers for plus : E --> E
        self.fc1 = nn.Linear(dim_E, neurons)
        self.fc2 = nn.Linear(neurons, neurons)
        self.fc3 = nn.Linear(neurons, neurons)
        self.fc4 = nn.Linear(neurons, dim_E)

        # dropout layer
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        output = self.fc4(x)
        return output

In [ ]:
class InverseMorphism(nn.Module):
    def __init__ (self, name = 'Inverse E --> R^n', dim_E = 1, neurons = 6):
        
        print(f'[Model] name : {name}')
        print(f'[Model] dim E : {dim_E}')
        print(f'[Model] no. neurons per layers : {neurons}')
        super(InverseMorphism, self).__init__()
        # layers for plus : E --> E
        self.fc1 = nn.Linear(dim_E, neurons)
        self.fc2 = nn.Linear(neurons, neurons)
        self.fc3= nn.Linear(neurons, neurons)
        
        self.fc4 = nn.Linear(neurons, dim_E)
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        output = self.fc4(x)
        return output

In [ ]:
class LoiBinaire(nn.Module):
    def __init__ (self, name = 'Loi binaire ExE-->E', dim_E = 1, neurons = 6):
        
        print(f'[Model] name : {name}')
        print(f'[Model] dim E : {dim_E}')
        print(f'[Model] no. neurons per layers : {neurons}')
        super(LoiBinaire, self).__init__()
        # layers for plus : ExE --> E
        self.fc1 = nn.Linear(2 * dim_E, neurons)
        self.fc2 = nn.Linear(neurons, neurons)
        self.fc3 = nn.Linear(neurons, neurons)
        self.fc4 = nn.Linear(neurons, dim_E)
    def forward(self, x, y):
        z = torch.cat([x,y], axis=1) # [K,d], [K,d] ---> [K, 2*d]
        z = self.fc1(z)
        z = self.fc2(z)
        z = self.fc3(z)
        output = self.fc4(z)
        return output
# scalaire product of structure

In [ ]:
class LoiScalaire(nn.Module):
    def __init__ (self, name = 'Loi Scalaire RxE-->E', dim_E = 1, neurons = 6):
        
        print(f'[Model] name : {name}')
        print(f'[Model] dim E : {dim_E}')
        print(f'[Model] no. neurons per layers : {neurons}')
        super(LoiScalaire, self).__init__()
        # layers for scaler : KxE --> E
        

        self.fc1 = nn.Linear(dim_E, neurons)
        self.fc2 = nn.Linear(neurons, neurons)
        self.fc3 = nn.Linear(neurons, neurons)
        self.fc4 = nn.Linear(neurons, dim_E)
        
        # alpha est un  scalaire,  dim_E est la dimension de l'espace E
        
    def forward(self, alpha, x):
        z = alpha * x # [K,1], [K,d] ---> [K, d]
        z = self.fc1(z)
        z = self.fc2(z)
        z = self.fc3(z)
        output = self.fc4(z)
        return output

In [ ]:
class Vect_space(nn.Module):
    def __init__ (self, K,  dim_E = 1 , neurons = 6 , name = 'Groupe (E,+)'):
        super(Vect_space, self).__init__()
        self.f    = Morphism(dim_E = dim_E, neurons = neurons)
        self.fi   = InverseMorphism(dim_E = dim_E, neurons = neurons)
        self.plus = LoiBinaire(dim_E = dim_E, neurons = neurons)
        self.scalaire = LoiScalaire(dim_E = dim_E, neurons = neurons)
        # losses
        self.loss_1 = lambda x, y : torch.linalg.vector_norm(self.plus(x , y) - self.f( self.fi(x) + self.fi(y)) )**2
        self.loss_2 = lambda alpha, x : torch.linalg.vector_norm(self.scalaire(alpha , x) - self.f( alpha*self.fi(x)) )**2

        #  Total loss can be weighted 
        self.loss = lambda x, y, alpha : self.loss_1(x, y) + self.loss_2(alpha, x)
        
    def train(self, X, Y,alpha, optimizer, epoch):
        self.f.train()
        self.fi.train()
        self.plus.train()
        self.scalaire.train()
        losses=[]
        for i in range(epoch):
            L1 = self.loss_1(X, Y)
            L2 = self.loss_2(alpha, X)
            loss = L1 + L2
            #loss = loss.mean()
            if i % 200 == 0:
               print('Epoch {}/{} -\t Loss 1: {:.6f}\t Loss 2: {:.6f}\t Total Loss: {:.6f}'.format(i, epoch, L1.item(), L2.item(), loss.item()))
            
            loss.backward(retain_graph=True)
            optimizer.step()
            optimizer.zero_grad()
            losses.append(loss.item())
        return losses

In [ ]:
#import numpy as np  
def line(K, epsilon ):
    
    X = 0.3*torch.randn(K, 2).requires_grad_(False)
    Y = 0.3*torch.randn(K, 2).requires_grad_(False)
    alpha = torch.randn(K, 1).requires_grad_(False)
    X[:,1] = X[:,0] + epsilon * torch.sin(X[:,0] / epislon ) # **3 + a*X[:,0] + b
    Y[:,1] = Y[:,0] + epsilon * torch.sin(Y[:,0] / epislon) #**3 + a*Y[:,0] + b
    return X, Y, alpha

In [ ]:
K = 2000
epislon = 0.1
X,Y,alpha = line(K, epislon)
dim = 2

# on initialise le vecteur space
G = Vect_space(K, dim_E = dim, neurons = 64)
# on initialise l'optimiseur

optimizer = optim.Adadelta(list(G.parameters()), lr=1e-3)
# la loss
losses = G.train(X,Y, alpha, optimizer, 1000)

plt.figure(figsize=(6, 4))
plt.plot(X[:,0], X[:,1], 'x', label='train X')
plt.title('Training Data X')
plt.legend()
plt.show()



# on affiche la loss 
plt.figure(figsize=(6, 4))
plt.plot(losses)
plt.title('Losses')
plt.show()

In [ ]:
K = 5
B = torch.rand((K, 2))
C = torch.rand((K, 2))





# Générer une valeur aléatoire pour B[0,0]
for i in range(K):
    B[i,1] = B[i,0] + epislon * torch.sin(B[i,0] / epislon )
    C[i,1] = C[i,0] + epislon * torch.sin(C[i,0] / epislon )



    
print(B)

print(C)


In [ ]:
import pandas as pd
# Convert B and C to numpy arrays
# Convert B and C to numpy arrays
B_np = B.numpy()
C_np = C.numpy()

# Create a DataFrame
df = pd.DataFrame({
    'B_x': B_np[:, 0],
    'B_y': B_np[:, 1],
    'C_x': C_np[:, 0],
    'C_y': C_np[:, 1],
})

print(df)



In [ ]:


# Supposons que X soit une matrice 2D de taille (n, 2)
# X = ...




plt.scatter(X[:, 0], X[:, 1], color='blue') 



colors_B = cm.rainbow(np.linspace(0, 1, B.shape[0]))
colors_C = cm.rainbow(np.linspace(0, 1, C.shape[0]))

plt.scatter(B[:, 0], B[:, 1], color=colors_B)
for i in range(B.shape[0]):
    plt.annotate(f'B_{i+1}', (B[i, 0], B[i, 1]))

plt.scatter(C[:, 0], C[:, 1], color=colors_C)
for i in range(C.shape[0]):
    plt.annotate(f'C_{i+1}', (C[i, 0], C[i, 1]))


plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

XXBC =  G.f(G.fi(B) + G.fi(C))
YYBC = G.plus(B, C)

for i in range(B.shape[0]):
    plt.figure()
    plt.scatter(X[:, 0], X[:, 1], color='blue') 
    plt.scatter(B[i, 0], B[i, 1], color='red', label=f'B_{i+1}')  
    plt.annotate(f'B_{i+1}', (B[i, 0], B[i, 1]))
    plt.scatter(C[i, 0], C[i, 1], color='green', label=f'C_{i+1}')
    plt.annotate(f'C_{i+1}', (C[i, 0], C[i, 1]))
    # Tracer le point XXBC[i]
    plt.scatter(XXBC[i, 0].detach().numpy(), XXBC[i, 1].detach().numpy(), color='orange', label=f'f.fi(B) + f.fi(C)')
    plt.annotate(f'f.fi(B) + f.fi(C)', (XXBC[i, 0].detach().numpy(), XXBC[i, 1].detach().numpy()))

    # Tracer le point YYBC[i]
    plt.scatter(YYBC[i, 0].detach().numpy(), YYBC[i, 1].detach().numpy(), color='pink', label=f'B⊕C')
    plt.annotate(f'B⊕C', (YYBC[i, 0].detach().numpy(), YYBC[i, 1].detach().numpy()))
    # Ajouter une légende au subplot
    plt.legend()
    plt.show()

In [ ]:
K = 5
B = torch.rand((K, 2))
alpha = torch.rand((K, 1))





# Générer une valeur aléatoire pour B[0,0]
for i in range(K):
    B[i,1] = B[i,0] + epislon * torch.sin(B[i,0] / epislon )



    
print(B)

print(alpha)


In [ ]:
import pandas as pd
# Convert B and C to numpy arrays
# Convert B and C to numpy arrays
B_np = B.numpy()
alpha_np = alpha.numpy()

# Create a DataFrame
df = pd.DataFrame({
    'B_x': B_np[:, 0],
    'B_y': B_np[:, 1],
    'alpha': C_np[:, 0],
})

print(df)



In [ ]:
XXBC =  G.f(alpha *G.fi(B))

YYBC = G.scalaire(alpha, B)

# print(XXBC)
# print(YYBC)



# calcul l'erreur entre les deux valeurs 
for i in range (len(XXBC)):
    print(f'Erreur {i+1} : {torch.linalg.vector_norm(XXBC[i] - YYBC[i])}')




XXBC_list = [x.detach().numpy() for x in XXBC]
YYBC_list = [y.detach().numpy() for y in YYBC]

# Calculer l'erreur pour chaque couple et la stocker dans une liste
erreur_list = [torch.linalg.vector_norm(XXBC[i] - YYBC[i]).item() for i in range(len(XXBC))]


# Convertir les erreurs en notation scientifique
erreur_list = ['{:.1e}'.format(erreur) for erreur in erreur_list]


# Ajouter les listes comme nouvelles colonnes dans le DataFrame
df['f(alpha * fi(B))'] = XXBC_list
df['alpha * B'] = YYBC_list
# Ajouter la colonne 'Erreur' à la fin du DataFrame
df['Erreur'] = erreur_list

df






In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

XXBC =  G.f(alpha *G.fi(B))

YYBC = G.scalaire(alpha, B)

for i in range(B.shape[0]):
    plt.figure()
    plt.scatter(X[:, 0], X[:, 1], color='blue', label='X') 
    plt.scatter(B[i, 0], B[i, 1], color='red', label=f'B_{i+1}')  
    plt.annotate(f'B_{i+1}', (B[i, 0], B[i, 1]))
    plt.scatter(C[i, 0], C[i, 1], color='green', label=f'C_{i+1}')
    plt.annotate(f'C_{i+1}', (C[i, 0], C[i, 1]))
    # Tracer le point XXBC[i]
    plt.scatter(XXBC[i, 0].detach().numpy(), XXBC[i, 1].detach().numpy(), color='orange', label=f'f(alpha * fi(B)')
    plt.annotate(f'f( alpha * fi(B))', (XXBC[i, 0].detach().numpy(), XXBC[i, 1].detach().numpy()))

    # Tracer le point YYBC[i]
    plt.scatter(YYBC[i, 0].detach().numpy(), YYBC[i, 1].detach().numpy(), color='pink', label=f'B⊕C')
    plt.annotate(f'B⊕C', (YYBC[i, 0].detach().numpy(), YYBC[i, 1].detach().numpy()))
    # Ajouter une légende au subplot
    plt.legend()
    plt.show()

In [ ]:
XXBC =  G.f(G.fi(B) + G.fi(C))

YYBC = G.plus(B, C)

# print(XXBC)
# print(YYBC)



# calcul l'erreur entre les deux valeurs 
for i in range (len(XXBC)):
    print(f'Erreur {i+1} : {torch.linalg.vector_norm(XXBC[i] - YYBC[i])}')




XXBC_list = [x.detach().numpy() for x in XXBC]
YYBC_list = [y.detach().numpy() for y in YYBC]

# Calculer l'erreur pour chaque couple et la stocker dans une liste
erreur_list = [torch.linalg.vector_norm(XXBC[i] - YYBC[i]).item() for i in range(len(XXBC))]


# Convertir les erreurs en notation scientifique
erreur_list = ['{:.1e}'.format(erreur) for erreur in erreur_list]


# Ajouter les listes comme nouvelles colonnes dans le DataFrame
df['f.fi(B) + f.fi(C)'] = XXBC_list
df['B⊕C'] = YYBC_list
# Ajouter la colonne 'Erreur' à la fin du DataFrame
df['Erreur'] = erreur_list

df




